In [3]:
pip install beautifulsoup4

     |████████████████████████████████| 112kB 6.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Segmenting and Clustering Neighborhoods
# First step is to intall Beautiful soup 4 according to recommendations

## We import the data from Wikipedia and convert it to a text file

In [4]:
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada:_M.html', 'w') as fo:
    fo.write(article)

In [5]:
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
article = open('List_of_postal_codes_of_Canada:_M.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:5] == ['Postal_Code', 'Borough', 'Neighborhood']:
        break
        
# Extract the columns we want and write to a semicolon-delimited text file.
with open('List_of_postal_codes_of_Canada:_M.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postal_Code, Borough, Neighborhood = [td.text.strip() for td in tds[:4]]
        # Wikipedia does something funny with country names containing
        # accented characters: extract the correct string form.
        #if '!' in country:
        #    country = country[country.index('!')+1:]
        print('; '.join([Postal_Code, Borough, Neighborhood]), file=fo)

In [6]:
import pandas as pd

# We create the dataframe and give the headers

In [9]:
df = pd.read_csv('List_of_postal_codes_of_Canada:_M.txt', sep=";", names = ["Postal_Name","Borough","Neighborhood"])

In [10]:
df.head()

,Postal_Name,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


# We drop the Not assigned rows

In [11]:
df.drop(df[df.Borough == " Not assigned"].index, inplace=True)

In [12]:
df.head()

,Postal_Name,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [13]:
df.shape

(103, 3)

In [14]:
contar = df.groupby(['Postal_Name']).count()
vali = contar[contar['Borough'] > 1]
vali.shape

(0, 2)

In [15]:
df = df.replace('/', ',', regex=True)

In [16]:
df.head()

,Postal_Name,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [17]:
df.shape

(103, 3)

# We create a new dataframe with coordinates

In [18]:
df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')


In [19]:
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# We join the do dataframes

In [20]:
pd_join = pd.merge(df, df_coordinates, left_on='Postal_Name', right_on='Postal Code', how='left').drop('Postal Code', axis=1)

pd_join.head()

,Postal_Name,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [22]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

In [23]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# We obtain the coordinates or toronto and create the map

In [24]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(pd_join['Latitude'], pd_join['Longitude'], pd_join['Borough'], pd_join['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [25]:
North_York_data = pd_join[pd_join['Borough'] == ' North York'].reset_index(drop=True)
North_York_data.head()

,Postal_Name,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [26]:
address = 'North York, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


# Map creation of North York

In [27]:
# create map of North York using latitude and longitude values
map_North_York = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(pd_join['Latitude'], pd_join['Longitude'], pd_join['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_North_York)  
    
map_North_York

In [28]:
CLIENT_ID = '1TPUE0CMO2FBONJ5C13PW1DSGWMNJJSTGZKY34PTOC1R0I3U' # your Foursquare ID
CLIENT_SECRET = '0WKMNH2PP34KYM30F3K4QNGKBYM4UWH4PH25CQYVRSOC2ODI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1TPUE0CMO2FBONJ5C13PW1DSGWMNJJSTGZKY34PTOC1R0I3U
CLIENT_SECRET:0WKMNH2PP34KYM30F3K4QNGKBYM4UWH4PH25CQYVRSOC2ODI


In [29]:
North_York_data.loc[0, 'Neighborhood']

' Parkwoods'

In [30]:
neighborhood_latitude = North_York_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = North_York_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = North_York_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of  Parkwoods are 43.7532586, -79.3296565.


In [31]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=1TPUE0CMO2FBONJ5C13PW1DSGWMNJJSTGZKY34PTOC1R0I3U&client_secret=0WKMNH2PP34KYM30F3K4QNGKBYM4UWH4PH25CQYVRSOC2ODI&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [32]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb189009fcb92001b16d8c0'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
North_York_venues = getNearbyVenues(names=North_York_data['Neighborhood'],
                                   latitudes=North_York_data['Latitude'],
                                   longitudes=North_York_data['Longitude']
                                  )


 Parkwoods
 Victoria Village
 Lawrence Manor , Lawrence Heights
 Don Mills
 Glencairn
 Don Mills
 Hillcrest Village
 Bathurst Manor , Wilson Heights , Downsview North
 Fairview , Henry Farm , Oriole
 Northwood Park , York University
 Bayview Village
 Downsview
 York Mills , Silver Hills
 Downsview
 North Park , Maple Leaf Park , Upwood Park
 Humber Summit
 Willowdale , Newtonbrook
 Downsview
 Bedford Park , Lawrence Manor East
 Humberlea , Emery
 Willowdale
 Downsview
 York Mills West
 Willowdale


In [44]:
print(North_York_venues.shape)
North_York_venues.head()

(238, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [45]:
North_York_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor , Wilson Heights , Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",23,23,23,23,23,23
Don Mills,27,27,27,27,27,27
Downsview,16,16,16,16,16,16
"Fairview , Henry Farm , Oriole",62,62,62,62,62,62
Glencairn,4,4,4,4,4,4
Hillcrest Village,5,5,5,5,5,5
Humber Summit,1,1,1,1,1,1


In [46]:
print('There are {} uniques categories.'.format(len(North_York_venues['Venue Category'].unique())))

There are 103 uniques categories.


In [47]:
# one hot encoding
North_York_onehot = pd.get_dummies(North_York_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
North_York_onehot['Neighborhood'] = North_York_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [North_York_onehot.columns[-1]] + list(North_York_onehot.columns[:-1])
North_York_onehot = North_York_onehot[fixed_columns]

North_York_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
North_York_onehot.shape

(238, 104)

In [49]:
North_York_grouped = North_York_onehot.groupby('Neighborhood').mean().reset_index()
North_York_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor , Wilson Heights , Downsview N...",0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.100000,0.000000,...,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000
1,Bayview Village,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,"Bedford Park , Lawrence Manor East",0.000000,0.0000,0.043478,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
3,Don Mills,0.000000,0.0000,0.000000,0.000000,0.074074,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
4,Downsview,0.000000,0.0625,0.000000,0.000000,0.000000,0.0625,0.000000,0.062500,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
5,"Fairview , Henry Farm , Oriole",0.000000,0.0000,0.016129,0.000000,0.016129,0.0000,0.016129,0.032258,0.016129,...,0.016129,0.000000,0.032258,0.000000,0.016129,0.032258,0.016129,0.00,0.000000,0.032258
6,Glencairn,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
7,Hillcrest Village,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
8,Humber Summit,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
9,"Humberlea , Emery",0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000


In [50]:
North_York_grouped.shape

(18, 104)

In [51]:
num_top_venues = 5

for hood in North_York_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = North_York_grouped[North_York_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Bathurst Manor , Wilson Heights , Downsview North----
              venue  freq
0       Coffee Shop  0.10
1              Bank  0.10
2     Deli / Bodega  0.05
3  Sushi Restaurant  0.05
4    Sandwich Place  0.05


---- Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


---- Bedford Park , Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09
2         Coffee Shop  0.09
3          Restaurant  0.09
4    Greek Restaurant  0.04


---- Don Mills----
                 venue  freq
0  Sporting Goods Shop  0.07
1           Restaurant  0.07
2     Asian Restaurant  0.07
3          Coffee Shop  0.07
4                  Gym  0.07


---- Downsview----
            venue  freq
0            Park  0.12
1   Grocery Store  0.12
2  Baseball Field  0.06
3  Discount Store  0.06
4   Shopping Mall  0.06


---- Fai

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = North_York_grouped['Neighborhood']

for ind in np.arange(North_York_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(North_York_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor , Wilson Heights , Downsview N...",Coffee Shop,Bank,Middle Eastern Restaurant,Deli / Bodega,Pharmacy,Pizza Place,Ice Cream Shop,Bridal Shop,Restaurant,Sandwich Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park , Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Greek Restaurant,Grocery Store,Hobby Shop,Indian Restaurant,Juice Bar,Liquor Store
3,Don Mills,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
4,Downsview,Park,Grocery Store,Baseball Field,Liquor Store,Business Service,Discount Store,Hotel,Shopping Mall,Gym / Fitness Center,Home Service


In [54]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 5

North_York_grouped_clustering = North_York_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(North_York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 2, 0, 3, 4], dtype=int32)

In [55]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

North_York_merged = North_York_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
North_York_merged = North_York_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

North_York_merged.head() # check the last columns!

,Postal_Name,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Intersection,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Food Truck,Food Court,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Gift Shop,Vietnamese Restaurant,Accessories Store
3,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,2.0,Park,Pizza Place,Pub,Japanese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping


In [56]:
North_York_merged = North_York_merged.fillna(0)
North_York_merged['Cluster Labels'] = North_York_merged['Cluster Labels'].astype(int)
North_York_merged.dtypes

North_York_merged.head(30)

,Postal_Name,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Food & Drink Shop,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Intersection,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Food Truck,Food Court,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Gift Shop,Vietnamese Restaurant,Accessories Store
3,M3B,North York,Don Mills,43.745906,-79.352188,0,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,2,Park,Pizza Place,Pub,Japanese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
5,M3C,North York,Don Mills,43.725900,-79.340923,0,Coffee Shop,Japanese Restaurant,Asian Restaurant,Restaurant,Gym,Sporting Goods Shop,Beer Store,Caribbean Restaurant,Shopping Mall,Dim Sum Restaurant
6,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Pool,Mediterranean Restaurant,Fast Food Restaurant,Dog Run,Dim Sum Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
7,M3H,North York,"Bathurst Manor , Wilson Heights , Downsview N...",43.754328,-79.442259,0,Coffee Shop,Bank,Middle Eastern Restaurant,Deli / Bodega,Pharmacy,Pizza Place,Ice Cream Shop,Bridal Shop,Restaurant,Sandwich Place
8,M2J,North York,"Fairview , Henry Farm , Oriole",43.778517,-79.346556,0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Japanese Restaurant,Bus Station,Bank,Food Court,Toy / Game Store
9,M3J,North York,"Northwood Park , York University",43.767980,-79.487262,0,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Coffee Shop,Women's Store,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


In [57]:
# create map

import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print()


In [58]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(North_York_merged['Latitude'], North_York_merged['Longitude'], North_York_merged['Neighborhood'], North_York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Above we can observe the map ann the clustering